In [1]:
import pandas as pd

from prophet import Prophet
import matplotlib.pyplot as plt

from sqlalchemy import create_engine
from sqlalchemy import event
from sqlalchemy import text
import os
from dotenv import load_dotenv
import sys
import warnings

/Users/tylerni/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
sys.path.append(os.path.abspath('..'))
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
from src.data_tool.connect_db import ConnectDB

In [4]:
db = ConnectDB()
engine = db.get_engine()

DB_USER and DB_PASSWORD are set


In [5]:
tables = db.show_tables()

In [6]:
tables

,Tables
0,capital_structure_debt
1,capital_structure_equity
2,capital_structure_summary
3,financial_ratios
4,fundamentals_annual
5,fundamentals_quarterly
6,identifiers
7,key_developments
8,security_daily


In [7]:
db.read_table('fundamentals_annual')

,gvkey,datadate,fyear,fyr,indfmt,consol,at,lt,ceq,ch,...,cogs,xsga,ebit,ebitda,ni,ib,xrd,capx,ticker,conm
0,117768,2025-01-31,2024,1,INDL,C,111601.000,32274.000,79327.000,8589.000,...,30775.000,16405.000,81453.000,83317.000,72880.000,72880.000,12914.000,3236.000,NVDA,NVIDIA CORP
1,339965,2025-01-31,2024,1,INDL,C,9033.938,6027.295,2999.929,2628.798,...,1072.277,3808.168,-1436.557,-1254.049,-1285.640,-1285.640,1783.379,75.712,SNOW,SNOWFLAKE INC
2,157855,2025-01-31,2024,1,INDL,C,102928.000,41755.000,61173.000,8848.000,...,6893.000,20685.000,7666.000,10317.000,6197.000,6197.000,5493.000,658.000,CRM,SALESFORCE INC
3,035005,2025-01-31,2024,1,INDL,C,10988.421,2053.337,8935.084,1349.380,...,999.690,2671.948,871.163,993.795,1010.238,1010.238,852.415,136.560,ZM,ZOOM COMMUNICATIONS INC
4,064768,2024-12-31,2024,12,INDL,C,624894.000,338924.000,285970.000,78779.000,...,294221.000,242315.000,69438.000,101423.000,59248.000,59248.000,88544.000,82999.000,AMZN,AMAZON.COM INC
5,006008,2024-12-31,2024,12,INDL,C,196485.000,91453.000,99270.000,8249.000,...,24566.000,21864.000,-4708.000,6671.000,-18756.000,-18756.000,16546.000,23944.000,INTC,INTEL CORP
6,023650,2024-12-31,2024,12,INDL,C,13924.000,2366.000,11558.000,1498.000,...,4396.000,3428.000,1020.000,1056.000,2019.000,2019.000,1367.000,19.000,SHOP,SHOPIFY INC
7,024616,2024-12-31,2024,12,INDL,C,81611.000,61194.000,20417.000,6561.000,...,16433.000,8569.000,5763.000,6795.000,4147.000,4147.000,1500.000,683.000,PYPL,PAYPAL HOLDINGS INC
8,036910,2024-12-31,2024,12,INDL,C,6340.884,1246.477,5003.275,2098.524,...,534.403,1989.114,310.403,341.990,462.190,462.190,507.878,12.634,PLTR,PALANTIR TECHNOLOG INC
9,001161,2024-12-31,2024,12,INDL,C,69226.000,11658.000,57568.000,3787.000,...,11594.000,9053.000,2290.000,5138.000,1641.000,1641.000,6456.000,636.000,AMD,ADVANCED MICRO DEVICES
